In [1]:
import json 
import pandas as pd 
import numpy as np
import re

pd.set_option('display.max_colwidth', None)
pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_json('data\\raw\\train-v2.0.json')

In [3]:
df_1 = pd.json_normalize(data=df['data'], record_path=['paragraphs'])

In [5]:
#create dataframe with columns of interest

df_2 = pd.DataFrame(columns=['question','plausible_answers','answers','context'])

for i in range(df_1.shape[0]):
    a = pd.DataFrame(df_1.iloc[i,0])
    a['context'] = df_1.iloc[i,1]
    try:
        a= a[['plausible_answers','answers','question','context']]
    except:
        a= a[['answers','question','context']]

    df_2=pd.concat([df_2,a])

    
    

In [6]:
#separate by certainty

df_certain = df_2.loc[df_2['plausible_answers'].isna()]
df_uncertain = df_2.loc[df_2['plausible_answers'].notna()]

In [7]:
#get answer position

def get_pos_ans(texto):
    return int(re.findall('(\d+)(?!.*\d)', texto)[0])

df_certain['answers']=df_certain['answers'].astype(str).apply(get_pos_ans)
df_uncertain['plausible_answers']=df_uncertain['plausible_answers'].astype(str).apply(get_pos_ans)

In [8]:
#insert answer flag
df_certain["context"] = [(t[:s] + '°' + t[s:]) for s, t in zip(df_certain["answers"], df_certain["context"])]
df_uncertain["context"] = [(t[:s] + '°' + t[s:]) for s, t in zip(df_uncertain["plausible_answers"], df_uncertain["context"])]


In [9]:
#removing unnecessary columns
df_certain = df_certain[['question','context']]
df_uncertain = df_uncertain[['question','context']]

In [ ]:
#text pre-processing

#read stopwords

file1 = open('data\\raw\\stopwords.csv', 'r')
stopwords_txt = file1.read().splitlines()
stopwords = []

for word in stopwords_txt:
    stopwords.append(word)

def limpieza(texto):
    
    #lower
    salida=texto.lower()
    
    #remove stopwords
    remove = '|'.join(stopwords)
    regex = re.compile(r'\b('+remove+r')\b', flags=re.IGNORECASE)
    salida = regex.sub("", salida)

    #remove unnecessary punctuation (will affect word matching)
    regex = r"[^\s\w\d\.\°]" 
    salida = re.sub(regex, '', salida)

    

    return salida




df_certain['context']=df_certain['context'].apply(limpieza)
df_uncertain['context']=df_uncertain['context'].apply(limpieza)
df_certain['question']=df_certain['question'].apply(limpieza)
df_uncertain['question']=df_uncertain['question'].apply(limpieza)


In [ ]:
#split into sentences
def spliteo(texto):

    salida=texto.split(sep='.')

    return salida


df_certain['context']=df_certain['context'].apply(spliteo)
df_uncertain['context']=df_uncertain['context'].apply(spliteo)

In [15]:
#unnest
df_certain=df_certain.explode('context')
df_uncertain=df_certain.explode('context')

In [16]:
#reading flag, eliminating flag
df_certain['res']=0
df_uncertain['res']=0

df_certain.loc[df_certain['context'].str.contains('°'),'res']=1
df_certain['context']=df_certain['context'].str.replace('°', '')
df_uncertain.loc[df_uncertain['context'].str.contains('°'),'res']=1
df_uncertain['context']=df_uncertain['context'].str.replace('°', '')

In [17]:
#strings to string lists

df_certain['context']=df_certain['context'].str.split(" ")
df_uncertain['context']=df_certain['context'].str.split(" ")

In [19]:
df_certain['question']=df_certain['question'].str.split(" ")
df_uncertain['question']=df_certain['question'].str.split(" ")

In [20]:
#jaccard
df_certain["jaccard"] = [(len(list(set(s) & set(t)))/len(list(set(s) | set(t)))) for s, t in zip(df_certain["question"], df_certain["context"])]
df_uncertain["jaccard"] = [(len(list(set(s) & set(t)))/len(list(set(s) | set(t)))) for s, t in zip(df_certain["question"], df_certain["context"])]

In [21]:
#removing unnecesary column
df_train_certain = df_certain[['res','jaccard']]
df_train_uncertain = df_uncertain[['res','jaccard']]

In [22]:
#export
df_train_certain.to_csv(r'data/results/df_train_certain.csv')
df_train_uncertain.to_csv(r'data/results/df_train_uncertain.csv')